# 04. LangGraph Planning Workflow Basics

Build a minimal planning graph without external API calls.


In [ ]:
from __future__ import annotations

import json
import os
import math
import random
import statistics
from pathlib import Path


def find_project_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / 'README.md').exists() and (candidate / 'main_langgraph.py').exists():
            return candidate
    return start


PROJECT_ROOT = find_project_root(Path.cwd().resolve())
os.chdir(PROJECT_ROOT)
print('PROJECT_ROOT =', PROJECT_ROOT)


In [ ]:
try:
    from langgraph.graph import StateGraph, START, END
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'langgraph>=0.2.0'])
    from langgraph.graph import StateGraph, START, END

from typing import TypedDict


In [ ]:
class MiniState(TypedDict, total=False):
    topic: str
    plan: list[str]
    findings: list[str]
    report: str


def plan_node(state: MiniState) -> MiniState:
    return {'plan': [f"search:{state['topic']}", 'summarize:findings']}


def execute_node(state: MiniState) -> MiniState:
    findings = [f"mock_result_for_{step}" for step in state.get('plan', [])]
    return {'findings': findings}


def synth_node(state: MiniState) -> MiniState:
    report = f"Report for {state['topic']} with {len(state.get('findings', []))} findings"
    return {'report': report}


In [ ]:
builder = StateGraph(MiniState)
builder.add_node('plan', plan_node)
builder.add_node('execute', execute_node)
builder.add_node('synthesize', synth_node)
builder.add_edge(START, 'plan')
builder.add_edge('plan', 'execute')
builder.add_edge('execute', 'synthesize')
builder.add_edge('synthesize', END)

app = builder.compile()
out = app.invoke({'topic': 'agent planning'})
print(out)

assert 'report' in out
assert 'agent planning' in out['report']
print('LangGraph mini workflow passed.')
